In [28]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pandas as pd
from qwikidata.entity import WikidataItem
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.utils import dump_entities_to_json
import pywikibot
import requests

In [61]:
def get_country_name(Q_COUNTRY):
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()
    item = pywikibot.ItemPage(repo, Q_COUNTRY)
    item_dict = item.get()
    if "P17" in item_dict["claims"]:
        if "en" in item_dict["labels"]:
            return item_dict["labels"]["en"]

In [66]:
def get_country_from_website(Q_WEBSITE):
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()
    item = pywikibot.ItemPage(repo, Q_WEBSITE)
    
    if not item.isRedirectPage():
        item_dict = item.get()
        if "P17" in item_dict["claims"]:
            clm_list = item_dict["claims"]["P17"]
            for clm in clm_list:
                clm_trgt = clm.getTarget()   

                return clm_trgt.text["labels"]["en"]

def get_identifier(website):
    params = dict (
            action='wbsearchentities',
            format='json',
            language='en',
            uselang='en',
            type='item',
            search=website
            )

    response = requests.get('https://www.wikidata.org/w/api.php?', params).json()
    if response.get('search'):
        return response.get('search')[0]['id']
    
def get_country_list_news(url_list, url_end_dic):
    country_website = dict()
  
    for index, url in enumerate(url_list):
        country =  None
        
        url_ending = url.split('.')[-1]
        if url_ending in url_end_dic:
            country = url_end_dic[url_ending]
        
        else:
            Q_WEBSITE = get_identifier(url)
            if Q_WEBSITE:
                country = get_country_from_website(Q_WEBSITE)

            if Q_WEBSITE is None or country is None:
                url_try_list = url.split('.')
                url_try = max(url_try_list, key=len)
                Q_WEBSITE = get_identifier(url_try)
                if Q_WEBSITE:
                    country = get_country_from_website(Q_WEBSITE)
    

        if country:
            country_website[url] = country
            print(country)
            
    return country_website

In [23]:
#Create website_country dictionary and save as json

file = open("C:/Users/jurri/Downloads/country_url_end.txt", "r")
url_end_dic = dict()
for index, line in enumerate(file):
    url_end_dic[line[:-1].split('\t')[1][1:]] = line[:-1].split('\t')[0]
    
URLS_FILE = open("C:/Users/jurri/Downloads/unique_urls.txt", "r")
unique_url_list = []
for index, line in enumerate(URLS_FILE):
    unique_url_list.append(line[:-1])
    
website_country_dic = get_country_list_news(unique_url_list, url_end_dic)

with open('Country_website.json', 'w') as fp:
    json.dump(website_country_dic, fp, sort_keys=True, indent=4)

South Africa
People's Republic of China
South Africa
United States of America
United States of America
United States of America
South Africa
India
Australia
Singapore
Kenya
United States of America
Ireland
Gabon
Australia
United States of America
United States of America
United States of America
United States of America
Australia
South Africa
Uruguay
Kenya
Australia
India
Australia
Australia
South Africa
Australia
Tuvalu
United States of America
Canada
Tanzania
Ghana
Australia
South Africa
United States of America
France
Australia
United States of America
Australia
Philippines
New Zealand
Pakistan
Australia
Canada
India
United States of America
Australia
Bangladesh
France
Australia
Canada
United States of America
United States of America
Pakistan
Kenya
New Zealand
Ireland
Canada
Australia
United States of America
United States of America
United States of America
Pakistan
Australia
New Zealand
India
Wales
United States of America
United States of America
Australia
Hong Kong
Australia
Wa

{'citizen.co.za': 'South Africa',
 'atimes.com': "People's Republic of China",
 'ecr.co.za': 'South Africa',
 'breitbart.com': 'United States of America',
 'washingtontimes.com': 'United States of America',
 'truthout.org': 'United States of America',
 'timeslive.co.za': 'South Africa',
 'moneycontrol.com': 'India',
 'byronnews.com.au': 'Australia',
 'straitstimes.com': 'Singapore',
 'nation.co.ke': 'Kenya',
 'qz.com': 'United States of America',
 'rte.ie': 'Ireland',
 'fool.com': 'Gabon',
 'frasercoastchronicle.com.au': 'Australia',
 'newsweek.com': 'United States of America',
 'europe.newsweek.com': 'United States of America',
 'latimes.com': 'United States of America',
 'sfgate.com': 'United States of America',
 'news.com.au': 'Australia',
 'ewn.co.za': 'South Africa',
 'en.mercopress.com': 'Uruguay',
 'theeastafrican.co.ke': 'Kenya',
 'smh.com.au': 'Australia',
 'ndtv.com': 'India',
 'sunshinecoastdaily.com.au': 'Australia',
 'thenewdaily.com.au': 'Australia',
 'afr.com': 'South Af

In [ ]:
#Create q_code_country dictionary and save as json
Q_file = open("country_q.txt", "r")
q_nat = []
for index, line in enumerate(Q_file):
    q_nat.append(line[:-1])
    
countries = dict()
for index, q in enumerate(q_nat):
    country = get_country_name(q)
    if country is not None:
        countries[q] = country
    
    if index%1000  ==0:
        print(index)
print(countries)

with open('Country_Q_code.json', 'w') as fp:
    json.dump(countries, fp, sort_keys=True, indent=4)

0


1000
